In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
# importing important libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
# importing the csv file from GitHub
url='https://raw.githubusercontent.com/rishmish0806/Capstone/refs/heads/main/dataset.csv'
df=pd.read_csv(url)

In [ ]:
# creating 14 DataFrames for each Location
df.SystemCodeNumber.unique()

df_locations=[df[df['SystemCodeNumber']==loc] for loc in df.SystemCodeNumber.unique()]
num_locs=len(df_locations)


In [ ]:
# creating TimeStamp feature in every DataSet
for df_i in df_locations:
  df_i['Timestamp'] = pd.to_datetime(df_i['LastUpdatedDate'] + ' ' + df_i['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

  # Sorting the DataFrame by the new 'Timestamp' column and reset the index
  df_i= df_i.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
  # encoding the traffic condition and 'Vehicle-Weight-Type'

for df_i in df_locations:
  encoding_dict1={'low':1,'average':2,'high':3}

  df_i.TrafficConditionNearby=df_i['TrafficConditionNearby'].map(encoding_dict1)
  df_i.head()

  df_i['VehicleType'].unique()

  encoding_dict2={'cycle':1,'bike':2,'car':3,'truck':4}
  df_i['VehicleType']=df_i['VehicleType'].map(encoding_dict2)
  df_i.head()
num_locs=len(df_locations)

In [ ]:
idx=0
a=1
b=1.5
c=1.2
d=1.3
e=5
for df_i in df_locations:



  df_i['Demand']=a*(df_i['Occupancy']/df_i['Capacity'])+b*df_i['QueueLength']-c*df_i['TrafficConditionNearby']+d*df_i['IsSpecialDay']+e*df_i['VehicleType']

  # Creating subset properly -- I use .copy() to ensure that df_subset is independent of df_i
  df_subset = df_i[['Timestamp','Demand']].copy()
  df_subset.to_csv(f"parking_stream{idx}.csv", index=False)
  idx+=1

In [ ]:
# print(pd.read_csv("parking_stream.csv").columns.tolist())

In [ ]:
# creating Schema
class ParkingSchema(pw.Schema):
    Timestamp:str
    Demand:float

In [ ]:
# creating mock-stream for different locations
data_locations=[pw.demo.replay_csv(f"parking_stream{i}.csv", schema=ParkingSchema, input_rate=1000) for i in range(num_locs)]

In [ ]:
# creating date_time Format and adding columns for time and day in a new data_frame

fmt = "%Y-%m-%d %H:%M:%S"


data_with_time_locations = [
    data.with_columns(
        t=pw.this.Timestamp.dt.strptime(fmt),  # Full timestamp
        day_dt=pw.this.Timestamp.dt.strptime(fmt).dt.floor("1d"),  # Date at midnight
        Demand=pw.this.Demand
    )
    for data in data_locations
]

In [ ]:
# here I have created delta_window in two steps
# in the first step I have created a daily_stats -- in which I compute demand_max and demand_min for a day for each location, which is later used in normalization
# in the second step I have created a delta-window which uses paramters from daily_stats -- to compute dynamic pricing


Base_Price=10

daily_stats = [
    data_with_time.windowby(
        pw.this.day_dt,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        day_dt=pw.reducers.any(pw.this.day_dt),
        demand_max=pw.reducers.max(pw.this.Demand),
        demand_min=pw.reducers.min(pw.this.Demand)
    )
    for data_with_time in data_with_time_locations
]



delta_window_locations = [
    data_with_time.join(
        stats,
        pw.left.day_dt == pw.right.day_dt
    ).select(
        t=pw.left.t,
        price=Base_Price * (1 + (pw.left.Demand - pw.right.demand_min) /
                         (pw.right.demand_max - pw.right.demand_min + 1e-6))
    )
    for data_with_time, stats in zip(data_with_time_locations, daily_stats)
]
pn.extension()


In [ ]:
# defining price plotter function
def price_plotter(source,title_of_plt):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=title_of_plt,
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=3, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

In [ ]:
# creating array of plots for different locations
plots = []

for i, source in enumerate(delta_window_locations):
    viz = source.plot(lambda src: price_plotter(src, f"Lot {i+1} Parking Price"), sorting_col="t")
    plots.append(viz)

dashboard = pn.Column(*plots, sizing_mode="stretch_width")

dashboard.servable("Multi-Plot Dashboard")


In [ ]:
# running the stream
%%capture --no-display
pw.run()